# Troubleshooting Agent chat for an online customer service scenario

<img src="media/autogen-troubleshooting.png" alt="Image description" width="700">

This notebook uses the following agents:

1. __User proxy__: Represents the human user who provides an initial complaint to customer service.
2. __Troubleshooter__: Troubleshooting agent responsible for analyzing the data from various sources to identify the potential problem. Troubleshooter uses general guidance on problem resolution, but needs to have the agency and capability of findng the solution amidst confusing and conflicting information.
3. __Data engineer__: Retrieves the needed data from the correct data sources to enable the troubleshooting process

## Setup
Install pyautogen with llm option:
```bash
pip install "pyautogen[lmm]>=0.2.3"
```

In [8]:
from IPython import get_ipython
from typing_extensions import Annotated
from dotenv import load_dotenv, find_dotenv

import autogen
from autogen.cache import Cache
import os

load_dotenv(find_dotenv())

config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o"],
    },
)

settings = {
    'host': os.getenv('ACCOUNT_HOST'),
    'master_key': os.getenv('ACCOUNT_KEY'),
    'database_id': os.getenv('COSMOS_DATABASE'),
    'container_id': os.getenv('COSMOS_CONTAINER'),
    'container_id_product': os.getenv('COSMOS_CONTAINER_PRODUCT'),
}

for key, value in settings.items():
    if value is None:
        raise ValueError(f"Missing environment variable for {key}")

In [26]:
troubleshooter_prompt = """
An experienced telecommunications troubleshooting expert. Troubleshoot the customer issue. Ask the data engineer for: 
1. Customer information based on customer id
2. Product information based on product id which you can only get by retrieving customer information first
3. Mass outage information based on the customer address which you can only get by retrieving customer information first

DO NOT PROVIDE USER INFORMATION FROM THE DATABASE DIRECTLY TO THE USER. 
Analyze the data carefully and identify the root cause of the issue. Think carefully, step-by-step and pay attention to anything unusual in the data.
If you need more information, ask the user. Provide a concise, clear and polite analysis to the user along with recommended mitigation issues. OMIT THE USER INFORMATION IRRELEVANT TO THE ISSUE IN YOUR ANSWERS.
Probe the user to give feedback on the analysis and recommendations.
Do make assumptions if necessary. Clarify with the user what exactly is the issue and what they are looking for, i.e "internet is slow" could mean many things, so make it clear.
"""

data_engineer_prompt = """
Data engineer, working with an experienced troubleshooter. 
DO NOT INTERACT WITH THE USER. DO NOT PROVIDE INFORMATION FROM THE DATABASE DIRECTLY TO THE USER. Only provide the data to the troubleshooter.
DO NOT ENGAGE IN ANY SORT OF CONVERSATION OR ANALYSIS BEYOND DATA RETRIEVAL. DO not say anything besides executing the commands.
"""

In [27]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

llm_config = {"config_list": config_list}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="ALWAYS",
    code_execution_config=False
)
troubleshooter = autogen.ConversableAgent(
    name="troubleshooter",
    system_message= troubleshooter_prompt,
    llm_config=llm_config,
)

temp_dir = tempfile.TemporaryDirectory()

executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim", 
    timeout=10, 
    work_dir=temp_dir.name,  
)
data_engineer = autogen.AssistantAgent(
    name="data_engineer",
    system_message=data_engineer_prompt,
    llm_config=llm_config,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER"
)


groupchat = autogen.GroupChat(agents=[user_proxy, troubleshooter, data_engineer], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


In [28]:
import json
import jsonlines
import requests
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import json

HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']
CONTAINER_ID_PRODUCT = settings['container_id_product']

client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY})
db = client.get_database_client(DATABASE_ID)
container = db.get_container_client(CONTAINER_ID)
container_product = db.get_container_client(CONTAINER_ID_PRODUCT)

@data_engineer.register_for_execution()
@troubleshooter.register_for_llm(name="get_customer_information", description="Retrieve customer data from the database")
def get_customer_info(customer_id: str) -> str:
    try:
        query = f"SELECT * FROM c WHERE c.id = '{customer_id}'"
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        
        if items:
            customer_info = items[0]
            return json.dumps(customer_info, default=str)
        else:
            error_message = {'error': f"Customer with ID '{customer_id}' not found."}
            return json.dumps(error_message)
    
    except exceptions.CosmosHttpResponseError as e:
        error_message = {'error': str(e)}
        return json.dumps(error_message)
    

@data_engineer.register_for_execution()
@troubleshooter.register_for_llm(name="get_product_information", description="Retrieve product data from the database")
def get_product_info(product_id: str) -> str:
    try:
        query = f"SELECT * FROM c WHERE c.ProductId = '{product_id}'"
        items = list(container_product.query_items(query=query, enable_cross_partition_query=True))
        
        if items:
            product_info = items[0]
            return json.dumps(product_info, default=str)
        else:
            error_message = {'error': f"Product with ID '{product_id}' not found."}
            return json.dumps(error_message)
    
    except exceptions.CosmosHttpResponseError as e:
        error_message = {'error': str(e)}
        return json.dumps(error_message)


  

# In our case a dummy outage.jsonl file is used to simulate the outage information
@data_engineer.register_for_execution()  
@troubleshooter.register_for_llm(name="get_outage_info", description="Retrieve the mass outage information for a customer's city")  
def get_outage_info(customer_address: str) -> str:  
    city = customer_address.split(', ')[1]  
      
    try:  
        with jsonlines.open('../utils/outage.jsonl') as reader:  
            for obj in reader:  
                if obj['city'] == city:  
                    return json.dumps(obj)  
    except FileNotFoundError:  
        return json.dumps({  
            'status': 'error',  
            'message': 'Outage information file not found.',  
            'affected_services': [],  
            'estimated_resolution_time': None  
        })  
      
    default_message = {  
        'status': 'unknown',  
        'message': f"No outage information available for {city}.",  
        'affected_services': [],  
        'estimated_resolution_time': None  
    }  
    return json.dumps(default_message)  


@data_engineer.register_for_execution()  
@troubleshooter.register_for_llm(name="restart_customer_router", description="Force restart the customer's router. Can only be executed with an explicit user consent and after all the other solutions have been considered.")  
def restart_customer_router(customer_id: str) -> str:  
    default_message = {  
        'status': 'Router successfully restarted'  
    }  
    return json.dumps(default_message)  



In [30]:
user_proxy.initiate_chat(
    manager, message="I'm experiencing issues with my internet connection. My customer id is Customer1. Can you help me troubleshoot?"
)

User_proxy (to chat_manager):

I'm experiencing issues with my internet connection. My customer id is Customer1. Can you help me troubleshoot?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
troubleshooter (to chat_manager):

***** Suggested tool call (call_P7QLgpj0I18twVUxYsGqNtCR): get_customer_information *****
Arguments: 
{"customer_id":"Customer1"}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_customer_information...
data_engineer (to chat_manager):

data_engineer (to chat_manager):

***** Response from calling tool (call_P7QLgpj0I18twVUxYsGqNtCR) *****
{"id": "Customer1", "FullName": "Olivia Davis", "Age": 52, "Address": "818 I Street, Chicago", "ServiceTier": "Gold", "InternetConnectionType": "Cable", "ContractTerm": 36, "MonthlyBill": 51.2, "PhoneNumber": "+1

ChatResult(chat_id=None, chat_history=[{'content': "I'm experiencing issues with my internet connection. My customer id is Customer1. Can you help me troubleshoot?", 'role': 'assistant'}, {'content': '', 'tool_calls': [{'id': 'call_P7QLgpj0I18twVUxYsGqNtCR', 'function': {'arguments': '{"customer_id":"Customer1"}', 'name': 'get_customer_information'}, 'type': 'function'}], 'name': 'troubleshooter', 'role': 'assistant'}, {'content': '{"id": "Customer1", "FullName": "Olivia Davis", "Age": 52, "Address": "818 I Street, Chicago", "ServiceTier": "Gold", "InternetConnectionType": "Cable", "ContractTerm": 36, "MonthlyBill": 51.2, "PhoneNumber": "+17239642979", "Email": "olivia.davis@email1.com", "ConnectionStatus": "Active", "ComplaintHistory": "Last month", "product_id": 101, "product_name": "Internet Package - Basic", "start_date": "2022-01-15", "service_id": "INT12345", "contract_id": "CTR67890", "status": "active", "last_billed_date": "2023-10-01", "next_billing_date": "2023-11-01", "data_